MachineLeaningCourse vol.10
# データの前処理

機械学習にはデータの質と有益な情報が何よりも欠かせません。なぜなら、データセット次第でモデルの精度がまったく違ってくるからです。

したがって、機械学習のアルゴリズムにデータを入れる前に、綺麗な形にデータを成形しなければいけません。これを__データの前処理__と呼びます。

### 欠損値の取り扱い
実際のデータでは、データセットのサンプルの値が一部欠損している場合がよくあります。こうした欠損値は、データが空欄であったり、__NaN__(Not a Number)などの文字列が入っています。

欠損値がある状態だと、機械学習のアルゴリズムは正しく動作しないため、データの前処理はとても大切な作業です。

In [2]:
# 欠損値を含むサンプルデータの作成
import pandas as pd
from io import StringIO

data ="""
A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0, 11.0, 12.0
"""

df = pd.read_csv(StringIO(data))

In [3]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


上記の様に、欠損値はNaNと表示されます。

DataFrameオブジェクトが大きい場合、データをすべて表示して欠損値を目視で確認していくのは大変です。

この場合は、isnullメソッドを使って、欠損値をカウントすることができます。

In [4]:
# 各特徴量の欠損値をカウント

df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

では、この欠損値を同処理したらいいのでしょうか？

大きく分けて手法は２つです。
- 欠損値が含まれている特徴量またはサンプルを取り除く
- 欠損値を補完する


#### 欠損値が含まれている特徴量またはサンプルを取り除く

In [5]:
# 欠損値を含む行を削除

df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
# 欠損値を含むサンプルを削除

df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [7]:
# すべての列がNanである行だけを削除

df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
# 非NaN値が4つ未満の行を削除

df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [12]:
# 任意の特徴量にNaNが含まれている場合にその列を削除

df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


#### 欠損値を補完する
上記の方法で欠損値を削除していくと、欠損値が多く含まれている場合や、そもそもデータの数が少ない場合などは、データセット数が少なくなってしまい、精度の高いモデルができない可能性が高いです。

このような場合、他のデータセットから欠損値を推測して補関する方法がとられます。
補完の方法は__平均値補完__と__回帰補完__があります。


#### 平均値補完
平均値補完では、単に結束値を持つ列または行の平均値と置き換えます。

scikit-learnにはこの処理を実装する便利なImputerクラスがあります。

In [18]:
from sklearn.preprocessing import Imputer

imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
## strategyには、以下が使用できます
# 'mean'   :  平均値
# 'median'  :  中央値
# 'most_frequent'  :  再頻出値

# axis=0 : 列、　axis=1 : 行

imr = imr.fit(df)
impured_data = imr.transform(df.values)
print(impured_data)

[[  1.    2.    3.    4. ]
 [  5.    6.    7.5   8. ]
 [ 10.   11.   12.    6. ]]


#### 回帰補完
Pandasには データの補間 を行うための interpolate( )メソッドがあります。

【公式ドキュメント】
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.interpolate.html

In [25]:
df.interpolate(axis=1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0
2,10.0,11.0,12.0,12.0


ドキュメントを見ると補間の手法が様々実装されてるのが分かります。
デフォルトでは線型補間によりデータを補完しますが、並び順に意味のあるデータでないと線形補間は意味がありません。

データに合わせた、補完法を考えるのが大切です。

### カテゴリデータの処理

現実のカテゴリデータに関しては、__順序特徴量__と__名義特徴量__を区別する必要があります。

__順序特徴量__とは、順序付けが可能なデータのことです。例えば、シャツのサイズはXL>L>M>Sのように順序を定義できるものです。

__名義特徴量__は、順序付けができないデータです。例えば、青いシャツと赤いシャツには大小関係を定義できません。

In [27]:
# サンプルデータの生成

df = pd.DataFrame([
    ['blue', 'XL', 120, 'class1'],
    ['red', 'S', 80, 'class2'],
    ['yellow', 'M', 100, 'class1'],
    ['green', 'L', 110, 'class2']
])

df. columns = ['color', 'size', 'price', 'classlabel']

In [28]:
df

,color,size,price,classlabel
0,blue,XL,120,class1
1,red,S,80,class2
2,yellow,M,100,class1
3,green,L,110,class2


#### 順序特徴量のマッピング
機械学習アルゴリズムに順序特徴量を正しく解釈させるには、カテゴリ文字列の値を整数に変換する必要があります。

In [29]:
# シャツのサイズと整数を対応させるディクショナリを生成
size_mapping = {'XL': 4, 'L': 3, 'M':2,'S':1}

df['size'] = df['size'].map(size_mapping)

In [30]:
df

,color,size,price,classlabel
0,blue,4,120,class1
1,red,1,80,class2
2,yellow,2,100,class1
3,green,3,110,class2


#### クラスラベルのマッピング
機械学習アルゴリズムは、クラスラベルが整数値としてエンコードされていることを要求しています。

In [34]:
import numpy as np

# クラスラベルと整数を対応させるディクショナリを生成
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
print(class_mapping)

{'class2': 1, 'class1': 0}


In [36]:
df['classlabel'] = df['classlabel'].map(class_mapping)

In [37]:
df

,color,size,price,classlabel
0,blue,4,120,0
1,red,1,80,1
2,yellow,2,100,0
3,green,3,110,1


#### 名義特徴量でのone-hotエンコーディング
名義特徴量は順序付けができないため、今まで見てきたようなやり方ではうまくいきません。

すなわち、blue, red, yellow, greenに０〜３を割り当てるという方法では、学習するときに色に順序付けがあると判定されてしまします。

この問題を回避する一般的な手法は、__one-hotエンコーディング__と呼ばれるものです。
この手法は、各ラベルをベクトルで表現します。
![](https://s3.amazonaws.com/ai-standard/pic10-1.png)

In [39]:
# pandasでone-hotエンコーディングを使い、ダミー変数を作成
pd.get_dummies(df['color'])

,blue,green,red,yellow
0,1,0,0,0
1,0,0,1,0
2,0,0,0,1
3,0,1,0,0
